Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

## Imports

In [1]:
from random import choices, choice, random, randrange
from dataclasses import dataclass
from copy import copy
from tqdm.auto import tqdm

import lab9_lib

## EA Strategy

### Hyper-parameters

In [2]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 100000
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .1
GENOTYPE_SIZE = 1000
PROBLEM_SIZE = 1
N_GENERATIONS = 10

### Initializations

In [3]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

### Classes and methods

In [4]:
@dataclass
class Individual:
    genotype: list[int]
    fitness: float


def select_parent(pop):
    """
    Selects a parent from the population using tournament selection.

    Parameters:
        pop (list[Individual]): The population of individuals.

    Returns:
        Individual: The selected parent.

    """

    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def mutate(ind: Individual) -> Individual:
    """
    Mutates an individual by flipping a random bit in its genotype.

    Parameters:
        ind (Individual): The individual to mutate.

    Returns:
        Individual: The mutated individual.

    """

    offspring = copy(ind)
    pos = randrange(0, GENOTYPE_SIZE)

    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None

    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    """
    Performs one-cut crossover between two individuals.

    Parameters:
        ind1 (Individual): The first individual.
        ind2 (Individual): The second individual.

    Returns:
        Individual: The offspring produced by one-cut crossover.
    """
    cut_point = randrange(0, GENOTYPE_SIZE)
    offspring = Individual(
        fitness=None, genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    )
    assert len(offspring.genotype) == GENOTYPE_SIZE
    return offspring

### Population generation

In [5]:
population = [
    Individual(genotype=choices([0, 1], k=GENOTYPE_SIZE), fitness=None)
    for _ in range(POPULATION_SIZE)
]

for ind in population:
    ind.fitness = fitness(ind.genotype)

population

[Individual(genotype=[1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 

### Offsprings generation

In [6]:
with tqdm(total=N_GENERATIONS * OFFSPRING_SIZE) as pbar:
    for generation in range(N_GENERATIONS):
        offspring = list()

        for counter in range(OFFSPRING_SIZE):
            # Mutation
            if random() < MUTATION_PROBABILITY:
                p = select_parent(population)
                o = mutate(p)
            # Crossover
            else:
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = one_cut_xover(p1, p2)

            offspring.append(o)

            pbar.update(1)

        # Evaluate fitness of offspring
        for i in offspring:
            i.fitness = fitness(i.genotype)

        # Add offspring to population and select top individuals
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        print(population[0].fitness)

        # Check if optimal solution is found
        if population[0].fitness == 1.0:
            break

print(fitness.calls)

  0%|          | 0/1000000 [00:00<?, ?it/s]

0.569
0.594
0.616
0.63
0.648
0.668
0.675
0.689
0.697
0.706
1000050


In [7]:
population

[Individual(genotype=[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 

## Random strategy

In [8]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
print(fitness.calls)

11100000001101101000110011000110101110011000110010: 9.13%
01010110101000100101101011110110001111001110001111: 23.33%
01011010100101000000010010111010000010101110110011: 15.34%
10010011001010101010111000000001011001111111001001: 7.33%
00010010001001010110100010011101001010000101001100: 17.56%
01011100110010111011001111010110000110101110110010: 9.11%
11110101101000110000110101110011110011110010011110: 23.33%
00011100100111011000100100011000000110011000000101: 11.56%
01100011000111110101111001110100111110010001011011: 31.33%
01000100100001100000110010101011100101001101111110: 7.33%
10
